# DATA ACQUISITION 

In [ ]:
import requests

def get_cat_fact():
    url = "https://catfact.ninja/fact"
    
    try:
        response = requests.get(url, timeout=5)  
        response.raise_for_status()  
        
        data = response.json()          
        fact = data.get("fact", "No fact found.")

        print("A nice fact about cats:")
        print(f"- {fact}")
    
    except requests.exceptions.HTTPError:
        print("HTTP Error.")
    except requests.exceptions.ConnectionError:
        print("Connection Error.")
    except requests.exceptions.Timeout:
        print("limit timeout exceeded.")
    except Exception as e:
        print(f"Unexpected error: {e}")

if __name__ == "__main__":
    get_cat_fact()


In [ ]:
def get_cat_facts(num_facts=3):
    url = "https://catfact.ninja/facts"
    params = {
        "limit": num_facts   
    }
    
    try:
        response = requests.get(url, params=params, timeout=5)
        response.raise_for_status()    
        data = response.json()
        print(f"Here  {num_facts} facts over cats:\n")
        for i, fact in enumerate(data.get("data", []), start=1):
            print(f"{i}. {fact['fact']}\n")
    
    except Exception as e:
        print(f"API eror: {e}")

if __name__ == "__main__":
    get_cat_facts(5)  


# Exercise

In [ ]:
## Advice Slip search

import requests

def search_advice(keyword: str):
    url = f"https://api.adviceslip.com/advice/search/{keyword}"
    try:
        print(f"Searching advice for '{keyword}'...")
        resp = requests.get(url, timeout=5)
        resp.raise_for_status()
        data = resp.json()

        slips = data.get("slips")
        if not slips:
            print(f"⚠️ No advice found for '{keyword}'.")
            return

        print(f"🔎 Advice results for '{keyword}':")
        for s in slips:
            print("•", s["advice"])
    except requests.exceptions.Timeout:
        print("Request timed out.")
    except requests.exceptions.HTTPError as e:
        print("HTTP error:", e)
    except Exception as e:
        print("Unexpected error:", e)

if __name__ == "__main__":
    kw ="life"
    search_advice(kw )


In [ ]:
## Geocoding and Weather Data

import csv
import requests
from statistics import mean

def geocode_city(city: str):
    url = "https://geocoding-api.open-meteo.com/v1/search"
    resp = requests.get(url, params={"name": city, "count": 1}, timeout=8)
    resp.raise_for_status()
    data = resp.json()
    results = data.get("results", [])
    if not results:
        raise ValueError(f"City '{city}' not found.")
    top = results[0]
    return top["latitude"], top["longitude"], top.get("name"), top.get("country")

def fetch_hourly_temps(lat: float, lon: float):
    url = "https://api.open-meteo.com/v1/forecast"
    params = {"latitude": lat, "longitude": lon, "hourly": "temperature_2m"}
    resp = requests.get(url, params=params, timeout=8)
    resp.raise_for_status()
    data = resp.json()
    hourly = data.get("hourly", {})
    times = hourly.get("time", [])
    temps = hourly.get("temperature_2m", [])
    return list(zip(times, temps))

def save_csv(rows, path="hourly_temps.csv"):
    with open(path, "w", newline="", encoding="utf-8") as f:
        w = csv.writer(f)
        w.writerow(["time", "temperature_2m"])
        w.writerows(rows)
    return path

if __name__ == "__main__":
    city =  "Singapore"
    lat, lon, name, country = geocode_city(city)
    rows = fetch_hourly_temps(lat, lon)
    temps = [t for _, t in rows]
    print(f"📍 {name}, {country} at ({lat:.3f}, {lon:.3f})")
    print(f"Hours fetched: {len(rows)} | Avg temp: {mean(temps):.1f}°C | Min: {min(temps):.1f} | Max: {max(temps):.1f}")
    out = save_csv(rows)
    print(f"💾 Saved CSV to {out}")
